<a href="https://colab.research.google.com/github/sriramkumar25/GenAIProjects/blob/main/Question_Answering_System_Using_Dense_Retrieval_%2B_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering System Using Dense Retrieval + BERT

## Overview
This system combines **Dense Retrieval** with **BERT** to extract answers from a knowledge base. It first converts documents into dense embeddings using **SentenceTransformers** and stores them in a **FAISS** index for fast retrieval. Then, **BERT** is used to extract the most relevant answer from the retrieved documents.

## Features
- **Dense Retrieval**: Uses **SentenceTransformers** for document embeddings and **FAISS** for efficient retrieval.
- **BERT for QA**: Utilizes the pre-trained BERT model to extract answers from retrieved documents.
- **Customizable**: Supports a variety of document sources for answering different types of queries.

## Tools & Technologies
- **SentenceTransformers**: For creating dense vector representations of documents.
- **FAISS**: For fast similarity-based document retrieval.
- **BERT**: For answer extraction from the retrieved documents.
- **Python** (transformers, faiss, sentence-transformers)

## Workflow
1. Convert documents into dense embeddings.
2. Index the embeddings in a **FAISS** index.
3. Use a query to retrieve the most relevant documents.
4. Apply **BERT** to extract the answer from the top documents.


In [5]:
!pip install faiss-gpu sentence-transformers transformers torch

In [6]:
import faiss
from sentence_transformers import SentenceTransformer
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

In [7]:
# Sample knowledge base
knowledge_base =[
    "The capital of Japan is Tokyo.",
    "The longest river in the world is the Nile.",
    "The largest ocean on Earth is the Pacific Ocean.",
    "Mount Everest is the highest mountain on Earth.",
    "The currency of the United Kingdom is the Pound Sterling.",
    "The fastest land animal is the cheetah.",
    "The human body has 23 pairs of chromosomes.",
    "The Great Wall of China is over 13,000 miles long.",
    "The Earth revolves around the Sun once every 365.25 days.",
    "The Eiffel Tower is located in Paris, France.",
    "The capital of Australia is Canberra.",
    "The smallest planet in our solar system is Mercury.",
    "The longest mountain range in the world is the Andes.",
    "The most spoken language in the world is Mandarin Chinese.",
    "The capital of Spain is Madrid."
]


In [8]:
# Step 1: Create a SentenceTransformer model for generating document embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 2: Generate embeddings for the knowledge base
document_embeddings = model.encode(knowledge_base)

# Step 3: Create a FAISS index
dimension = document_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings)

# Load pre-trained BERT model and tokenizer for Question Answering
model_qa = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Function to retrieve the most relevant document
def retrieve_top_document(query, faiss_index, knowledge_base):
    query_embedding = model.encode([query])  # Encode the query
    distances, indices = faiss_index.search(query_embedding, 1)  # Retrieve top document
    return knowledge_base[indices[0][0]], distances[0][0]


In [12]:
# Function for answering questions based on retrieved context
def answer_question(context, question):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model_qa(**inputs)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer_tokens = inputs.input_ids[0][start_index:end_index+1]
    answer = tokenizer.decode(answer_tokens)
    return answer


In [16]:
while True:
    query = input("Enter your question: ")
    if(query == "exit"):
        print(f"Exiting, Goodbye!")
        break
    # Retrieve the most relevant document from the knowledge base
    context, score = retrieve_top_document(query, faiss_index, knowledge_base)
    print(f"Retrieved Context: {context} (Score: {score})")

    # Generate an answer using BERT
    answer = answer_question(context, query)
    print(f"Answer: {answer}")

Enter your question: what is the smallest planet in our solar system
Retrieved Context: The smallest planet in our solar system is Mercury. (Score: 0.23081859946250916)
Answer: mercury
Enter your question: how long is great wall of china
Retrieved Context: The Great Wall of China is over 13,000 miles long. (Score: 0.3596913516521454)
Answer: over 13, 000 miles
Enter your question: exit
Exiting, Goodbye!
